<a href="https://colab.research.google.com/github/mammadmaheri7/core_GNN/blob/main/GNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Python Enviroment for GNN:

- https://docs.dgl.ai/index.html

- https://graphneural.network

- https://www.pyg.org

- http://snap.stanford.edu/gnn-design/

In [1]:
! pip install dgl

     |████████████████████████████████| 4.4 MB 5.4 MB/s 


In [2]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

import itertools
import numpy as np
import scipy.sparse as sp

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


Using backend: pytorch


# Node Classification

The Cora dataset consists of 2708 scientific publications classified into one of seven classes. The citation network consists of 5429 links. Each publication in the dataset is described by a 0/1-valued word vector indicating the absence/presence of the corresponding word from the dictionary. The dictionary consists of 1433 unique words.

https://relational.fit.cvut.cz/dataset/CORA

In [3]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()
print('Number of categories:', dataset.num_classes)

Extracting file to /root/.dgl/cora_v2
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
Number of categories: 7


In [4]:
g = dataset[0]

In [5]:
print('Node features')
print(g.ndata)
print('Edge features')
print(g.edata)

Node features
{'train_mask': tensor([ True,  True,  True,  ..., False, False, False]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])}
Edge features
{}


In [6]:
g.ndata.keys()

dict_keys(['train_mask', 'val_mask', 'test_mask', 'label', 'feat'])

In [7]:
g.ndata['label']

tensor([3, 4, 4,  ..., 3, 3, 3])

In [8]:
g.edges()

(tensor([   0,    0,    0,  ..., 2707, 2707, 2707]),
 tensor([ 633, 1862, 2582,  ...,  598, 1473, 2706]))

In [9]:
g.out_degrees([0,1])

tensor([3, 3])

In [20]:
from dgl.nn import GraphConv

class GCN(nn.Module): # nn.Module is Base class for all neural network modules.
    # https://pytorch.org/docs/stable/generated/torch.nn.Module.html
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN,self).__init__()
        self.conv1 = GraphConv(in_feats,h_feats)
        self.conv2 = GraphConv(h_feats,num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

# Create the model with given dimensions
model = GCN(g.ndata['feat'].shape[1], 16, 7)

In [26]:
def train(g, model):

    features = g.ndata['feat']
    labels = g.ndata['label']

    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_test_acc = 0
    best_val_acc = 0

    for e in range(100):
        # Forward
        out = model(g,features)
       
        # Compute prediction
        pred = out.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(out[g.ndata['train_mask']], labels[g.ndata['train_mask']])

        # Compute accuracy on training/validation/test
        train_acc = (labels[g.ndata['train_mask']] == pred[g.ndata['train_mask']]).float().mean()
        val_acc = (labels[g.ndata['val_mask']] == pred[g.ndata['val_mask']]).float().mean()
        test_acc = (labels[g.ndata['test_mask']] == pred[g.ndata['test_mask']]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        best_val_acc = max(best_val_acc,val_acc)
        best_test_acc = max(best_test_acc,test_acc)

        # Backward
        # Sets the gradients of all optimized torch.Tensor s to zero.
        #https://pytorch.org/docs/stable/generated/torch.optim.Optimizer.zero_grad.html
        optimizer.zero_grad()
        loss.backward()
        
         # Performs a single optimization step (parameter update).
        #https://pytorch.org/docs/stable/generated/torch.optim.Optimizer.step.html
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))

In [27]:
train(g,model)

In epoch 0, loss: 1.946, val acc: 0.070 (best 0.070), test acc: 0.089 (best 0.089)
In epoch 5, loss: 1.893, val acc: 0.464 (best 0.464), test acc: 0.518 (best 0.518)
In epoch 10, loss: 1.816, val acc: 0.548 (best 0.572), test acc: 0.543 (best 0.558)
In epoch 15, loss: 1.715, val acc: 0.564 (best 0.572), test acc: 0.559 (best 0.559)
In epoch 20, loss: 1.589, val acc: 0.598 (best 0.598), test acc: 0.601 (best 0.601)
In epoch 25, loss: 1.444, val acc: 0.646 (best 0.646), test acc: 0.643 (best 0.643)
In epoch 30, loss: 1.283, val acc: 0.672 (best 0.672), test acc: 0.668 (best 0.668)
In epoch 35, loss: 1.117, val acc: 0.690 (best 0.690), test acc: 0.684 (best 0.684)
In epoch 40, loss: 0.951, val acc: 0.694 (best 0.694), test acc: 0.705 (best 0.705)
In epoch 45, loss: 0.795, val acc: 0.712 (best 0.712), test acc: 0.720 (best 0.721)
In epoch 50, loss: 0.654, val acc: 0.730 (best 0.730), test acc: 0.734 (best 0.734)
In epoch 55, loss: 0.531, val acc: 0.740 (best 0.740), test acc: 0.753 (best 0

you can find the spectral code of this example in:
https://github.com/danielegrattarola/spektral/blob/master/examples/node_prediction/citation_gcn.py